# AlexNet

Running AlexNet on fruits & vegetables dataset.

Model references:

- <https://papers.nips.cc/paper/2012/hash/c399862d3b9d6b76c8436e924a68c45b-Abstract.html>
- <https://arxiv.org/pdf/1404.5997>


In [1]:
from torchvision.models import alexnet, AlexNet_Weights

from cvision.data import get_fruits, VisionDataset

In [2]:
weights = AlexNet_Weights.IMAGENET1K_V1
model = alexnet(weights=weights)